In [1]:
!pip uninstall -y tensorflow

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1


In [2]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 37kB/s 


In [3]:
!pip install tqdm

In [4]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2021-02-08 16:06:57--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.210.128, 173.194.211.128, 173.194.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.210.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   241MB/s    in 0.3s    

2021-02-08 16:06:57 (241 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [5]:
import os

In [6]:
import zipfile

In [7]:
import numpy as np

In [8]:
import tensorflow as tf

In [9]:
import matplotlib.pyplot as plt

In [10]:
from tqdm import tqdm_notebook

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
%matplotlib inline

In [13]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [14]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [15]:
zip_object.extractall("./")

In [16]:
zip_object.close()

In [17]:
dataset_path_new = "./cats_and_dogs_filtered"

In [18]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

In [19]:
img_shape = (128, 128, 3)

In [20]:
base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape, 
                                               include_top = False,
                                               weights = "imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [21]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [22]:
base_model.trainable = False

In [23]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [24]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [25]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [26]:
prediction_layer = tf.keras.layers.Dense(units = 1, activation = "sigmoid")(global_average_layer)

In [27]:
model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [29]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.0001),
              loss="binary_crossentropy", metrics = ["accuracy"])

In [30]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [31]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")
valid_generator = data_gen_train.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [32]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
16/16 [==============================] - 49s 3s/step - loss: 0.6131 - accuracy: 0.6640 - val_loss: 0.5231 - val_accuracy: 0.7540
Epoch 2/5
16/16 [==============================] - 45s 3s/step - loss: 0.4974 - accuracy: 0.7772 - val_loss: 0.4578 - val_accuracy: 0.8150
Epoch 3/5
16/16 [==============================] - 46s 3s/step - loss: 0.4436 - accuracy: 0.8205 - val_loss: 0.4056 - val_accuracy: 0.8440
Epoch 4/5
16/16 [==============================] - 46s 3s/step - loss: 0.3950 - accuracy: 0.8487 - val_loss: 0.3626 - val_accuracy: 0.8770
Epoch 5/5
16/16 [==============================] - 52s 3s/step - loss: 0.3448 - accuracy: 0.8774 - val_loss: 0.3265 - val_accuracy: 0.8960


In [33]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [34]:
valid_accuracy

0.8960000276565552

In [35]:
base_model.trainable = True
len(base_model.layers)

154

In [36]:
fine_tuning_at = 100

In [37]:
for layer in base_model.layers[ : fine_tuning_at ]:
  layer.trainable = False

In [38]:
model.compile( optimizer = tf.keras.optimizers.RMSprop( learning_rate = 0.0001 ), loss = 'binary_crossentropy', metrics = ['accuracy'] )

In [39]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
16/16 [==============================] - 81s 5s/step - loss: 0.2457 - accuracy: 0.8898 - val_loss: 0.0734 - val_accuracy: 0.9750
Epoch 2/5
16/16 [==============================] - 72s 5s/step - loss: 0.0275 - accuracy: 0.9987 - val_loss: 0.0718 - val_accuracy: 0.9740
Epoch 3/5
16/16 [==============================] - 72s 5s/step - loss: 0.0120 - accuracy: 0.9984 - val_loss: 0.0864 - val_accuracy: 0.9710
Epoch 4/5
16/16 [==============================] - 72s 5s/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.1115 - val_accuracy: 0.9680
Epoch 5/5
16/16 [==============================] - 72s 5s/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.1344 - val_accuracy: 0.9690


In [40]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [41]:
valid_accuracy

0.968999981880188